In [1]:
#리뷰어 key unique 개수 확인
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd
import pickle

In [4]:
with open("review.pickle", "rb") as g:
    review = pickle.load(g) # 수집한 리뷰어 key 불러오기

review['리뷰자'].nunique()

5982

In [ ]:
df = review[['리뷰자']].reset_index(drop=True)
df.drop_duplicates(inplace=True) #리뷰자 고유값만 남기기
repeat_values = df['리뷰자'].repeat(20) #20번씩 반복
df = pd.DataFrame(repeat_values, columns=['리뷰자'])

df['type1'] = np.nan; df['type2'] = np.nan; df['type3'] = np.nan; df['type4'] = np.nan
df['브랜드'] = np.nan; df['상품'] = np.nan; df['평점'] = np.nan; df['작성일자'] = np.nan
df['본문'] = np.nan
df.reset_index(drop=True, inplace=True)

In [11]:
# free-proxy get
proxy = FreeProxy(rand = True, https = True).get()

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

#proxy server 사용
proxy_server = proxy.split('//')[1]
chrome_options.add_argument(f'--proxy-server={proxy_server}')
service = Service(executable_path=ChromeDriverManager().install())

In [12]:
#반복문으로 구현하기
base_url = 'https://www.oliveyoung.co.kr/store/mypage/getReviewerProfile.do?key={}'
driver = webdriver.Chrome(service=service, options=chrome_options)

# 유저 반복문 추가하기
for m,n in enumerate(df['리뷰자']):
    if pd.isnull(df.loc[m, '상품']): #크롤링 안 된 부분부터
        if m % 20 == 0: # 인덱스가 20의 배수인 경우만 
            url = base_url.format(n)
            try:
                driver.get(url)
                driver.implicitly_wait(5)
                for _ in range(2): #스크롤 2번 실행
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(1)

                resp = driver.page_source #현재 페이지 파싱
                soup = BeautifulSoup(resp, "html.parser")
                for i in range(0,20): #상품 20개 크롤링
                    try:
                        type = soup.find('ul','profile-keyword-list').find_all('li','list-item')
                        if len(type) == 4: #타입 정보가 전부 있을 경우
                            df.loc[m+i,'type1'] = type[0].get_text()
                            df.loc[m+i,'type2'] = type[1].get_text()
                            df.loc[m+i,'type3'] = type[2].get_text()
                            df.loc[m+i,'type4'] = type[3].get_text()
                        else: #타입 정보가 일부만 있을 경우
                            for j in range(len(type)):
                                df.loc[m+i, 'type{}'.format(j+1)] = type[j].get_text()
                    except AttributeError: #타입 정보가 없을 경우
                        df.iloc[m+i, 1:5] = np.nan #결측값으로 채우기
                    try:
                        df.loc[m+i, '브랜드'] = soup.select('p.rw-box-figcaption__brand')[i].get_text()
                        df.loc[m+i, '상품'] = soup.select('p.rw-box-figcaption__name')[i].get_text()
                        df.loc[m+i,'평점'] = soup.select('span.point')[i].get_text()
                        df.loc[m+i, '작성일자'] = soup.select('span.review_point_text')[i].get_text()
                        df.loc[m+i, '본문'] = soup.select('p.rw-box__description')[i].get_text()
                        df.loc[m+i, '랭킹'] = soup.select('div.profile-badge.on')[0].get_text()
                    except IndexError: #리뷰한 상품 개수가 20개보다 적을 경우
                        df.iloc[m+i, 5:] = np.nan #결측값으로 채우기
                if m % 1000 == 0:
                    print(m, "번째 크롤링 완료")
            except: #에러 발생 시 멈춤
                print(m, "번째에서 에러 발생")
                break

driver.quit()

0 번째 크롤링 완료
1000 번째 크롤링 완료
2000 번째 크롤링 완료
3000 번째 크롤링 완료
4000 번째 크롤링 완료
5000 번째 크롤링 완료
6000 번째 크롤링 완료
7000 번째 크롤링 완료
8000 번째 크롤링 완료


In [ ]:
#크롤링된 경우의 데이터프레임 생성
crawl_done = df[(df['상품'].isna() == False)]

In [ ]:
#프로필 비공개인 경우 : 크롤링된 경우 이전에 한해서
private_temp = df.iloc[:list(crawl_done.tail(1).index)[0]+1, :].copy()
private = private_temp[private_temp['상품'].isna() == True]

In [15]:
#크롤링된 경우 + 프로필 비공개인 경우 비율
print((crawl_done.shape[0] + private.shape[0]) / df.shape[0] * 100)

100.0


In [16]:
#크롤링된 경우 + 프로필 비공개인 경우 저장
save_df = df.iloc[:list(crawl_done.tail(1).index)[0]+1, :].copy()

#이전에 크롤링한 결과랑 concat
total_df = pd.concat([previous, save_df])

In [17]:
# 겹치는 거 있는지 확인
previous.merge(save_df)

,리뷰자,type1,type2,type3,type4,브랜드,상품,평점,작성일자,본문


In [18]:
# 크롤링한 결과 저장
with open("crawl_done.pickle", "wb") as g:
    pickle.dump(total_df, g)